# Setup

## Mounting drive files

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## All tensorflow related

In [3]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [4]:
!nvidia-smi

Thu Aug 19 20:16:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Import libraries

In [5]:
!pip install gpt-2-simple
%tensorflow_version 1.x
import gpt_2_simple as gpt2
import os
import requests
import tensorflow as tf
import pandas as pd
import re

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



# Model setup

In [6]:
model_name = "355M"
if not os.path.isdir(os.path.join("models", model_name)):
	print(f"Downloading {model_name} model...")
	gpt2.download_gpt2(model_name=model_name)   # model is saved into current directory under /models/124M/

In [7]:
checkpoint_dir ='/content/drive/MyDrive/proyectos/ia/recipie/cherckponts5'
path = '/content/drive/MyDrive/proyectos/ia/recipie/'
file_name = 'full_dataset'
file_path = path+file_name

prefix= '<|startoftext|>'
sufix ='<|endoftext|>'

In [8]:
##para cargar ultimo punto de guardado resetear entorno y correr 
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess,checkpoint_dir=checkpoint_dir,run_name='run1')

Loading checkpoint None


ValueError: ignored

# Model testing

In [9]:
tf.reset_default_graph()
sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              file_path+'2.txt',
              model_name=model_name,
              checkpoint_dir=checkpoint_dir,   
              steps=1000)   

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [01:19<00:00, 79.03s/it]


dataset has 15596259 tokens
Training...
[1 | 14.23] loss=1.86 avg=1.86
[2 | 17.47] loss=2.10 avg=1.98
[3 | 20.70] loss=1.96 avg=1.97
[4 | 23.93] loss=1.91 avg=1.96
[5 | 27.19] loss=1.78 avg=1.92
[6 | 30.43] loss=1.79 avg=1.90
[7 | 33.68] loss=1.79 avg=1.88
[8 | 36.92] loss=1.90 avg=1.89
[9 | 40.18] loss=1.62 avg=1.85
[10 | 43.41] loss=1.74 avg=1.84
[11 | 46.65] loss=1.95 avg=1.85
[12 | 49.88] loss=1.55 avg=1.83
[13 | 53.14] loss=1.69 avg=1.81
[14 | 56.39] loss=1.50 avg=1.79
[15 | 59.64] loss=1.77 avg=1.79
[16 | 62.88] loss=1.64 avg=1.78
[17 | 66.12] loss=1.89 avg=1.79
[18 | 69.38] loss=1.59 avg=1.77
[19 | 72.63] loss=1.76 avg=1.77
[20 | 75.85] loss=1.67 avg=1.77
[21 | 79.08] loss=1.65 avg=1.76
[22 | 82.32] loss=1.65 avg=1.76
[23 | 85.55] loss=1.71 avg=1.75
[24 | 88.78] loss=1.80 avg=1.76
[25 | 92.04] loss=2.19 avg=1.78
[26 | 95.27] loss=1.83 avg=1.78
[27 | 98.51] loss=1.63 avg=1.77
[28 | 101.72] loss=1.65 avg=1.77
[29 | 104.97] loss=1.60 avg=1.76
[30 | 108.20] loss=1.51 avg=1.75
[31 | 

# Model testing with specific ingredients

In [13]:
text = """INGREDIENTS
egg
nutella
chocolate
flour
strawberries
DIRECTIONS
"""

In [16]:
gpt2.generate(sess,prefix=text,truncate=sufix,checkpoint_dir=checkpoint_dir,nsamples=3)

INGREDIENTS
egg
nutella
chocolate
flour
strawberries
DIRECTIONS
Mix all ingredients together.
Roll in flour until rolled very well.
Place on platter and place in refrigerator.
TITLE
Chocolate Crunch

INGREDIENTS
egg
nutella
chocolate
flour
strawberries
DIRECTIONS
Put all ingredients in a large bowl. Bake at 375\u00b0 for about an hour.
TITLE
Perfect Pie

INGREDIENTS
egg
nutella
chocolate
flour
strawberries
DIRECTIONS
Beat egg, nutella and flour together until smooth.
Stir in flour and chocolate.
Bake at 350\u00b0 for 1 to 2 minutes.
TITLE
Chocolate Fudge Cake



In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

In [ ]:

ds = pd.read_csv(path + file_name+'.csv')
df = ds[['ingredients','directions','title']]
titles = df.keys()

## Saving results in a txt file

In [ ]:
with open(path+file_name+'4.txt', 'w+') as f:
  for i, row in df.iterrows():
    s=prefix+'\n'
    for x,item in enumerate(row):
      s2 = str(item)
      s2=re.sub('\"\s?,\s?\"','\n',s2)
      s2=re.sub('\"|\[|\]','',s2)
      s+=titles[x].upper()+'\n'+s2+'\n';
    s+=sufix+'\n'
    if(i%10000==0):
      print(s)
    f.write(str(s))
    if ((i+1 )% 500000 )==0 :
      break;
f.close()

<|startoftext|>
INGREDIENTS
1 c. firmly packed brown sugar
1/2 c. evaporated milk
1/2 tsp. vanilla
1/2 c. broken nuts (pecans)
2 Tbsp. butter or margarine
3 1/2 c. bite size shredded rice biscuits
DIRECTIONS
In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.
Stir over medium heat until mixture bubbles all over top.
Boil and stir 5 minutes more. Take off heat.
Stir in vanilla and cereal; mix well.
Using 2 teaspoons, drop and shape into 30 clusters on wax paper.
Let stand until firm, about 30 minutes.
TITLE
No-Bake Nut Cookies
<|endoftext|>

<|startoftext|>
INGREDIENTS
1 c. sugar
1/2 stick butter
1 tsp. vanilla
2 eggs
1 c. milk
end pieces of sandwich bread, cut up
1/2 c. raisins (if desired)
1/2 c. coconut (if desired)
DIRECTIONS
Cream sugar and butter.
Add eggs, vanilla and milk.
Stir. Add pieces of sandwich bread, enough to absorb the moisture but not enough to make dry.
Add raisins and coconut if desired.
Bake at 350\u00b0 until brown.
TITLE
B

In [ ]:
print(titles[1])

directions
